In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import cv2
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
TRAIN_DATA_PATH = "data/train_data"
#train_data folder has subfolders for each letter
#Each subfolder has training images in jpg format

NUM_OF_LETTERS = 10
IMAGE_SIZE = 50 # We'll be working with 50 * 50 pixel images
NUM_OF_CHANNELS = 1 # Grayscale
NUM_OF_TRAIN_IMAGES = 2000*NUM_OF_LETTERS
NUM_OF_TEST_IMAGES = NUM_OF_LETTERS
NUM_OF_DENSE_LAYER_NODES = (IMAGE_SIZE * IMAGE_SIZE) // 20

LABELS = ['A', 'C', 'E', 'H', 'I', 'L', 'O', 'U', 'V', 'W']
len(LABELS)

10

# Creating train / test data

In [3]:
def create_data(DATA_PATH):
    """returns list of [image_array, label_name].
    Keyword arguments:
    DATA_PATH -- Path of train/ test data
    """
    data = []
    for label in LABELS:
        path = os.path.join(DATA_PATH, label)
        label_name = LABELS.index(label)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                img_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))
                
                data.append([img_array, label_name]) # Stores pair as image array -> corresponding label name 
            except Exception as e:
                pass
            
    return data
        
training_data = create_data(TRAIN_DATA_PATH)        

In [4]:
def make_datasets(data):
    """returns pair of lists of X and y
    Keyword arguments:
    data -- list of [img_array, label_name]
    """
    x = []
    y = []
    for features, label in data:
        x.append(features)
        y.append(label)
    return x,y

X_train, y_train = make_datasets(training_data)
len(X_train)

20000

In [5]:
# Convert datasets to numpy arrays

X_train, y_train = np.array(X_train), np.array(y_train)


In [6]:
# Resizing images

X_train = X_train.reshape(NUM_OF_TRAIN_IMAGES, IMAGE_SIZE , IMAGE_SIZE, NUM_OF_CHANNELS)
print (X_train.shape)


(20000, 50, 50, 1)


In [7]:
# Normalizing as [0,1]


X_train = X_train.astype("float32")
X_train /= 255.0

In [8]:
X_train.shape

(20000, 50, 50, 1)

In [9]:
#one-hot encoding

y_train = to_categorical(y_train, NUM_OF_LETTERS)


In [10]:
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_train.shape

(20000, 50, 50, 1)

In [11]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


# Creating CNN model

In [12]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, padding = "same", input_shape = (IMAGE_SIZE, IMAGE_SIZE, NUM_OF_CHANNELS), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2),strides=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=3, activation="relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(NUM_OF_DENSE_LAYER_NODES, activation="relu"))
model.add(Dense(50, activation="relu"))

model.add(Dense(NUM_OF_LETTERS, activation = "softmax"))


# Compiling CNN Model

In [13]:
model.compile(
    optimizer="adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"])

In [14]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 128)      

# Training the model

In [15]:
history = model.fit(
    X_train, 
    y_train, 
    batch_size = 128,
    epochs = 4,
#     validation_split = 0.1,
    shuffle = True
)

Epoch 1/4
20000/20000 [==============================] - 26s 1ms/step - loss: 0.4424 - accuracy: 0.8536
Epoch 2/4
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0239 - accuracy: 0.9920
Epoch 3/4
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0196 - accuracy: 0.9937
Epoch 4/4
20000/20000 [==============================] - 25s 1ms/step - loss: 0.0067 - accuracy: 0.9980


In [ ]:
# saving the trained model

# from keras.models import load_model
# model.save('withbgmodelv_small_test.h5') 

In [17]:

# score = model.evaluate(X_test, y_test)
# print(score[0])
# print(score[1])

In [18]:
# TEST_DATA_PATH = "data_big/train_data"

In [19]:
# test_data = create_data(TEST_DATA_PATH)        

In [20]:
# X_test, y_test = make_datasets(test_data)


In [21]:
# X_test, y_test = np.array(X_test), np.array(y_test)


In [22]:
# X_test = X_test.reshape(30000, IMAGE_SIZE , IMAGE_SIZE, NUM_OF_CHANNELS)


In [23]:
# X_test = X_test.astype("float32")
# X_test /= 255.0

In [24]:
# y_test = to_categorical(y_test, NUM_OF_LETTERS)
